In [116]:
import os
import json
import re
import requests
import nltk
import pandas as pd
import numpy as np
from unidecode import unidecode
from pandas.parser import CParserError
from bs4 import BeautifulSoup
from xml.etree import ElementTree

Exploring API structure
------------

In [2]:
# Full request parameters. Two "all" parameters are allowed per request.
"""
http://wits.worldbank.org/API/V1/SDMX/V21/datasource/TRN/
reporter/{all|reporter code list}
/partner/{all|partner code list}
/product/{ALL|product code list}
/year/{All|Year List
/datatype/{reported|aveestimated}}"""
    
    
    
countries_url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/country/ALL"
nomenclature_url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/nomenclature/" #H0 through H4
products_url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/product/all" # nomenclaturecode is HS

# Country, product, nomenclature, partner list, 
# number preferential agreements, is specific duty expression estimated, last update
data_available_url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/dataavailability/" 

In [3]:
# US 840. Canada 124. EU 918. 710 South Africa
# Make sure partner is taken from Partner list in data availability
test_url = "http://wits.worldbank.org/API/V1/SDMX/V21/datasource/TRN/reporter/918/partner/710/product/ALL/year/2008/datatype/reported"

In [4]:
res = requests.get(test_url)

In [5]:
root = ElementTree.fromstring(res.content)
series = root.findall(".//Series")
obs = root.findall(".//Obs")
dataset = root.findall(".//Dataset")

In [6]:
first = series[0].findall(".//Obs")[0].attrib
attribute = series[0].attrib
combined = {**first,**attribute}
df = pd.DataFrame(columns=list(combined.keys()))

data = [{**s.findall(".//Obs")[0].attrib,**s.attrib}  for s in series]

df = df.append(data)

In [7]:
df.head()

,TIME_PERIOD,NBR_NA_LINES,TARIFFTYPE,PARTNER,TOTALNOOFLINES,DATATYPE,OBS_VALUE,REPORTER,MIN_RATE,NBR_MFN_LINES,MAX_RATE,OBS_VALUE_MEASURE,FREQ,SUM_OF_RATES,NOMENCODE,PRODUCTCODE,NBR_PREF_LINES
0,2008,0,PREF,710,3,Reported,0.400000015894572,918,0,1,1.20000004768372,SimpleAverage,A,1.20000004768372,H3,010110,2
1,2008,0,PREF,710,4,Reported,0.300000011920929,918,0,1,1.20000004768372,SimpleAverage,A,1.20000004768372,H3,010190,3
2,2008,1,PREF,710,2,Reported,0,918,0,1,0,SimpleAverage,A,0,H3,010391,1
3,2008,2,PREF,710,3,Reported,0,918,0,1,0,SimpleAverage,A,0,H3,010392,2
4,2008,2,PREF,710,3,Reported,0,918,0,1,0,SimpleAverage,A,0,H3,010410,2


In [8]:
df.describe()

,TIME_PERIOD,NBR_NA_LINES,TARIFFTYPE,PARTNER,TOTALNOOFLINES,DATATYPE,OBS_VALUE,REPORTER,MIN_RATE,NBR_MFN_LINES,MAX_RATE,OBS_VALUE_MEASURE,FREQ,SUM_OF_RATES,NOMENCODE,PRODUCTCODE,NBR_PREF_LINES
count,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562,3562
unique,1,27,1,1,44,1,101,1,44,25,65,1,1,104,1,3562,39
top,2008,0,PREF,710,1,Reported,0,918,0,0,0,SimpleAverage,A,0,H3,520532,1
freq,3562,3363,3562,3562,1561,3562,3306,3562,3349,2983,3306,3562,3562,3306,3562,1,1798


In [9]:
df[df.MAX_RATE==df.MIN_RATE].count()

TIME_PERIOD          3489
NBR_NA_LINES         3489
TARIFFTYPE           3489
PARTNER              3489
TOTALNOOFLINES       3489
DATATYPE             3489
OBS_VALUE            3489
REPORTER             3489
MIN_RATE             3489
NBR_MFN_LINES        3489
MAX_RATE             3489
OBS_VALUE_MEASURE    3489
FREQ                 3489
SUM_OF_RATES         3489
NOMENCODE            3489
PRODUCTCODE          3489
NBR_PREF_LINES       3489
dtype: int64

In [10]:
df[df.MAX_RATE=="0"].count()

TIME_PERIOD          3306
NBR_NA_LINES         3306
TARIFFTYPE           3306
PARTNER              3306
TOTALNOOFLINES       3306
DATATYPE             3306
OBS_VALUE            3306
REPORTER             3306
MIN_RATE             3306
NBR_MFN_LINES        3306
MAX_RATE             3306
OBS_VALUE_MEASURE    3306
FREQ                 3306
SUM_OF_RATES         3306
NOMENCODE            3306
PRODUCTCODE          3306
NBR_PREF_LINES       3306
dtype: int64

In [11]:
len(df[df.MAX_RATE==""]) # 96 missing or empty max rates

96

In [12]:
df[['NOMENCODE','PRODUCTCODE']]

,NOMENCODE,PRODUCTCODE
0,H3,010110
1,H3,010190
2,H3,010391
3,H3,010392
4,H3,010410
5,H3,010420
6,H3,010511
7,H3,010512
8,H3,010519
9,H3,010594


In [13]:
# WITS UNCTAD TRAINS provides graphical query interface with API-like url structure
gui_url = "http://wits.worldbank.org/tariff/trains/en/country/USA/year/2014/pagenumber/1/pageSize/134684"
res = requests.get(gui_url)

In [14]:
soup = BeautifulSoup(res.text,"html.parser")

In [15]:
scripts = soup.find_all("script")
len(scripts)

19

In [16]:

p = re.search("(localdata: \[)(.*)",scripts[7].text)
data = p.group(2)

In [17]:
fields = ['ProductCode','ProductDescription','Partner','PartnerName','AdValorem','MeasureName','NonAdValorem','AffectedPartners']
rows = [s+"," for s in data.split("},")]


output = []
for row in rows:
    r = {}
    for f in fields:
        s = re.search("(?<="+f+" :)(.*?)(?=,)",row)
        value = s.group(0).replace("\"","")
        #print(f+":"+value)
        r.update({f:value})
    output.append(r)
print(r)

{'MeasureName': ' Non-MFN duty rate for countries excluded from Most-Favoured- Nation (MFN) treatement  ', 'NonAdValorem': '   ', 'AffectedPartners': '   }\r', 'ProductCode': ' 97060000  ', 'PartnerName': ' Non-MFN Countries for USA: 2006  ', 'AdValorem': ' 0.00  ', 'ProductDescription': ' Antiques of an age exceeding one hundred years  ', 'Partner': ' N76  '}


In [18]:
f = "ProductCode"
s = re.search("(?<="+f+" :)(.*?)(?=,)",rows[0])
value = s.group(0).replace("\"","")
value

' 01012100  '

In [19]:
p = re.search('var totalRecords= ([0-9]{1,6})',scripts[7].text)
p.group(0)

'var totalRecords= 134684'

In [20]:
dat = pd.DataFrame(output)


In [21]:
nav_unique = dat.NonAdValorem.unique()
units = """cent_tariff: {<CD><NN>}
           cent_tariff: {<CD><JJ><NN>}
           cent_tariff: {<CD><NNS><DT>}
           dollar_tariff:{<$>?<CD>}
           dollar_tariff:{<VBZ><CD><NN>}
           content:{<IN><DT><NN>}
           content:{<IN><NN><NN>}
           content:{<IN><JJ><NNS>}
           content:{<IN><NN>}
           """ # 6.6 cents/kg
percent = "price:{<$>?<CD>}" # $ 1.64/kg
content = "<IN<NN><NN>" #on molybdenum content
extra_feature = "<IN><DT><NN>" # on the case

# value currency_unit weight_unit 
cp = nltk.RegexpParser(units)
for n in nav_unique:
    pos = nltk.pos_tag(nltk.word_tokenize(n))
    #print(cp.parse(pos))
    
    

In [22]:
dat[dat.NonAdValorem!='   ']

,AdValorem,AffectedPartners,MeasureName,NonAdValorem,Partner,PartnerName,ProductCode,ProductDescription
49,1.96,,Non-MFN duty rate for countries excluded from...,6.6 cents/kg,N76,Non-MFN Countries for USA: 2006,01022920,Cows imported specially for dairy purposes
50,0.30,,Most Favoured Nation duty rate treatement,1 cents/kg,000,World,01022940,Live cattle other than purebred or those impo...
51,0.12,,Australia - United States Free Trade Agreemen...,0.4 cents/kg,036,Australia,01022940,Live cattle other than purebred or those impo...
68,1.63,,Non-MFN duty rate for countries excluded from...,5.5 cents/kg,N76,Non-MFN Countries for USA: 2006,01022940,Live cattle other than purebred or those impo...
72,0.30,,Most Favoured Nation duty rate treatement,1 cents/kg,000,World,01023900,Live buffalo
73,0.12,,Australia - United States Free Trade Agreemen...,0.4 cents/kg,036,Australia,01023900,Live buffalo
90,1.64,,Non-MFN duty rate for countries excluded from...,5.5 cents/kg,N76,Non-MFN Countries for USA: 2006,01023900,Live buffalo
92,0.28,,Most Favoured Nation duty rate treatement,1 cents/kg,000,World,01029000,Live bovine animals
93,0.11,,Australia - United States Free Trade Agreemen...,0.4 cents/kg,036,Australia,01029000,Live bovine animals
110,1.53,,Non-MFN duty rate for countries excluded from...,5.5 cents/kg,N76,Non-MFN Countries for USA: 2006,01029000,Live bovine animals


Create dataset of all Non-ad valorem tariffs
--

In [23]:
data_dir = "D:/Users/cmarciniak/Documents/macmap/data/unctad/specific"
files = os.listdir(data_dir)

In [24]:
df = pd.DataFrame()
for f in files:
    try:
        file = os.path.join(data_dir,f)
        data = pd.read_csv(file,encoding='latin-1',dtype='object')
        subset = data[data.NonAdValorem!='   ']
        df = df.append(subset)
    except CParserError:
        print("Error ocurred in file: "+f)

Error ocurred in file: JOR2003.csv


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1616659 entries, 214 to 44156
Data columns (total 11 columns):
Unnamed: 0            1616659 non-null object
AdValorem             1616659 non-null object
AffectedPartners      1616659 non-null object
MeasureName           1616659 non-null object
NonAdValorem          1616659 non-null object
Partner               1616659 non-null object
PartnerName           1616659 non-null object
ProductCode           1616659 non-null object
ProductDescription    1616659 non-null object
Year                  1616659 non-null object
ISO3Code              1616659 non-null object
dtypes: object(11)
memory usage: 148.0+ MB


In [56]:
df['ProductCode'].str.len().value_counts()

11    932704
13    359273
12    167946
9      51230
16     50984
15     31938
10     14718
14      5799
17      2067
Name: ProductCode, dtype: int64

In [73]:
df['pc'] = df['ProductCode'].str.strip()
df['pc'].str.len().value_counts()


8     938793
10    356955
9     171404
6      51230
13     47517
12     28734
7      14718
11      5808
14      1500
Name: pc, dtype: int64

In [87]:
df[df['pc'].str.len()==12]

,Unnamed: 0,AdValorem,AffectedPartners,MeasureName,NonAdValorem,Partner,PartnerName,ProductCode,ProductDescription,Year,ISO3Code,pc
1206,1206,,,Most Favoured Nation duty rate treatement,$109.99 per m3,000,World,271000131000,MOTOR SPIRIT EXPORTED UNDER THE PROCESSING AG...,2001,BRB,271000131000
1207,1207,,,Most Favoured Nation duty rate treatement,$109.99 per m3,000,World,271000131090,MOTOR SPIRIT EXPORTED UNDER THE PROCESSING AG...,2001,BRB,271000131090
1208,1208,,,Most Favoured Nation duty rate treatement,$109.99 per m3,000,World,271000139000,OTHER MOTOR SPIRIT (GASOLINE),2001,BRB,271000139000
1209,1209,,,Most Favoured Nation duty rate treatement,$109.99 per m3,000,World,271000139090,OTHER MOTOR SPIRIT (UNLEADED GASOLINE),2001,BRB,271000139090
1210,1210,,,Most Favoured Nation duty rate treatement,$65.99 per m3,000,World,271000221000,ILLUMINATING KEROSENE EXPORTED UNDER THE PROC...,2001,BRB,271000221000
1211,1211,,,Most Favoured Nation duty rate treatement,$65.99 per m3,000,World,271000229000,Illuminating kerosene,2001,BRB,271000229000
1212,1212,,,Most Favoured Nation duty rate treatement,$65.99 per m3,000,World,271000231000,VAPOURISING OIL EXPORTED UNDER THE PROCESSING...,2001,BRB,271000231000
1213,1213,,,Most Favoured Nation duty rate treatement,$65.99 per m3,000,World,271000239000,OTHER VAPOURISING OIL OR WHITE SPIRIT,2001,BRB,271000239000
1214,1214,,,Most Favoured Nation duty rate treatement,$65.99 per m3,000,World,271000311010,AUTOMOTIVE DIESEL (PROCESSING AGREEMENT) FOR ...,2001,BRB,271000311010
1215,1215,,,Most Favoured Nation duty rate treatement,$65.99 per m3,000,World,271000311090,AUTOMOTIVE DIESEL (PROCESSING AGREEMENT) NOT ...,2001,BRB,271000311090


In [114]:
df.AdValorem.value_counts()

              478628
 15.00         19108
 30.00         17341
 20.00         16767
 10.00         16753
 5.00          14099
 0.00          14026
 25.00         12037
 5.60           7717
 40.00          6447
 35.00          6438
 24.00          5570
 9.00           4850
 60.00          4615
 2.50           4614
 27.00          3794
 2.00           3442
 6.00           3277
 80.00          2895
 12.00          2490
 50.00          2379
 11.25          2332
 8.30           2190
 18.00          2111
 100.00         1860
 0.18           1827
 0.07           1822
 12.50          1749
 0.01           1723
 1.00           1696
               ...  
 663.81            1
 167.33            1
 217.21            1
 146.17            1
 197.34            1
 4614.82           1
 395.53            1
 753.69            1
 190.73            1
 1030.13           1
 2497.10           1
 168.75            1
 559.60            1
 405.54            1
 171.93            1
 384.79            1
 136.05      

In [127]:
len(df)

1616659

In [126]:
len(df[df.AdValorem.str.isspace()])

478628

In [113]:
def lookup(key,dictionary):
    val = ''
    try:
        val = dictionary[key]
    except KeyError:
        pass
    return val

In [27]:
currency_codes = {
	'F': 'Aruban Florin',
    'AMD':'Armenian Dram',
	'BD': 'Local Currency', 
	'DH':'Arab Emirates Dirham',
	'AED':'Arab Emirates Dirham',
	'DLARES':'USD',
	'$':'USD',
	'USD':'USD',
	'EUR':'Euro',
    'EURO':'Euro',
    'BHD':'Bahraini Dinar',
	'BSD':'Bahamian Dollar', 
	'KM':'Bosnian Mark',
	'BZD':'Belize Dollar',
	'BMD':'Bermudan Dollar',
	'Bolivian':'Bolivian Boliviano',
	'C':'cents',
	'CENTS':'cents',
	'CFA':'Central African Franc',
	'FR.':'Local Franc',
	'SFR':'Swiss Franc',
	'RMB':'Chinese Yuan',
	'YUAN':'Chinese Yuan',
	'CNY':'Chinese Yuan',
	'NZD':'New Zealand Dollar',
	'CYP':'Cypriot Pound',
	'C£':'Cypriot Pound',
	'DA':'Algerian Dinar',
    'DR':'Dominican Peso',
	'EGP':'Egyptian Pound',
	'POUND':'Local Pound',
	'LE':'Egyptian Pound',
	'EGL':'Egyptian Pound',
	'ECU':'European Currency Unit',
	'FJD':'Fijian Dollar',
	'GMD':'Gambian Dalasi',
	'RS':'Local Rupee',
	'ISK':'Iceland Krona',
	'SDR':'Special Drawing Right',
	'KR':'Swedish Kroner',
	'NIS':'Israeli New Sheqel',
	'YEN':'Japanese Yen',
	'Y':'Japanese Yen',
	'KES':'Kenyan Shilling',
	'SHS.':'Kenyan Shilling',
    'SH':'Shilling',
	'XCD':'East Caribbean Dollar',
	'WON':'Korean Won',
	'LYH':'Libyan Dinar',
	'LYD':'Libyan Dinar',
	'DINAR':'Libyan Dinar',
	'S':'Peruvian Sole',
	'RM':'Malaysian Ringgit',
	'M$':'Malaysian Ringgit',
	'NGN':'Nigerian Naira',
	'NK':'Norwegian Kroner',
	'NOK':'Norwegian Kroner',
	'KRONE':'Norwegian Kroner',
	'BALBOA':'Panamanian Balboa',
	'SAR':'Saudi Riyal',
	'SR':'Saudi Riyal',
	'SGD':'Singaporean Dollar',
	'S$':'Singaporean Dollar',
	'SBD':'Solomon Island Dollar',
	'SIT':'Slovenian Tolar',
	'SR':'Saudi Riyal',
	'BHT':'Thai Baht',
	'BAHTS':'Thai Baht',
	'B':'Thai Baht',
	'AUD':'Australian Dollar',
    'A$':'Australian Dollar',
	'NT$':'New Taiwan Dollar',
	"BOLIVARES":"Bolivian Bolivar",
	"SAT$":"Samoan Tala",
	"ZAR":"South African Rand",
	"RAND":"South African Rand"
}

units = {
    'KILOGRAMO':'kg',
	'KG':'kg',
	'LI':'litre',
	'LITRE':'liter',
    'LITER':'liter',
	'DOZEN': 'dozen',
    'DOZ':'dozen',
	'HL':'hectoliter',
	'SQUARE METER':'square meter',
	'SQM':'square meter',
	'SQ.MTR.':'square meter',
	'SQ. METRE':'square meter',
	'TONNE':'tonne',
	'T':'tonne',
	'KILOLITRE':'kiloliter',
	'KL': 'kilolitre',
	'SZT':'Polish sztych',
    'GALLON':'gallon',
    'GAL.':'gallon',
	'PAR':'pair',
	'M²':'square meter',
    'PCS':'pieces',
    'CM':'centimeters',
    'TON':'tonne',
    'ITEMS':'items',
    'UNIT':'unit',
    'M2':'square meter',
    'SM3':'standard cubic meter',
    'KGS':'kg',
    'PCE':'piece',
    'PIECE':'piece',
    'KILO':'kg',
    'KW':'kilowatt',
    'IMP.GAL': 'imperial gallon',
    'DAL':'dekaliter',
    'PRS':'pairs',
    'L':'liter',
    'LAA':'liters absolute alcohol'
    
}

operators = {
	'but not less than': '>=',
	'OR': 'OR',
	'+': 'AND',
    'AND':'AND',
	'PLUS':'AND',
	'MAX.':'max',
	'MIN.':'min',
    'MIN':'min',
	'MINIMUM':'min',
	'MAKSIMUM':"max",
	'QUOTA':'quota',
	'HIGHER':'>'
}

In [45]:
nav_unique = df.NonAdValorem.unique()
patterns = """
        
           dollar_tariff:{<$><CD><NN>}
           dollar_tariff:{<VBZ><CD><NN>}
           numeric_component: {<CD><NN>}
           numeric_component: {<CD><NNP>}
           cent_tariff: {<CD><JJ><NN>}
           cent_tariff: {<CD><NNS><DT>}
           dollar_tariff:{<$>?<CD>}
           content:{<IN><DT><NN>}
           content:{<IN><NN><NN>}
           content:{<IN><JJ><NNS>}
           weight_unit:{<IN><NN>}
           weight_unit:{<NNP><NN>}
           qualifier:{<NNP>}
           """ # 6.6 cents/kg
percent = "price:{<$>?<CD>}" # $ 1.64/kg
content = "<IN><NN><NN>" #on molybdenum content
extra_feature = "<IN><DT><NN>" # on the case

# value currency_unit weight_unit 
cp = nltk.RegexpParser(patterns)
trees2 = []
for n in nav_unique:
    items = {}
    i = 1
    j = 1
    items['text'] = n
    n = n.replace("/"," ").upper()
    pos = nltk.pos_tag(nltk.word_tokenize(n))
    tree = cp.parse(pos)
    for subtree in tree.subtrees():
        if subtree.label()=="numeric_component" and subtree[1][0]=='%':
            items['advalorem_component'+str(i)] = subtree[0][0]
            i = i+1
        if subtree.label()=="numeric_component" and lookup(subtree[1][0],operators)!="":
            items['advalorem_component'+str(i)] = subtree[0][0]
            i = i+1
        if subtree.label()=="numeric_component" and subtree[1][0]!='%':
            items['value'+str(j)] = subtree[0][0]
            j = j+1
    for leaf in tree.leaves():
        if lookup(leaf[0],units)!="":
            items['unit'] = lookup(leaf[0],units)
        if lookup(leaf[0],currency_codes)!="":
            items['currency_unit'] = lookup(leaf[0],currency_codes)
        if lookup(leaf[0],operators)!="":
            items['operator'] = lookup(leaf[0],operators)
    trees2.append(items)
    

In [46]:
spec = pd.DataFrame(trees2)

In [47]:

spec[spec['advalorem_component1'].isnull()]

,advalorem_component1,advalorem_component2,advalorem_component3,advalorem_component4,advalorem_component5,currency_unit,operator,text,unit,value1,value2,value3,value4,value5,value6,value7,value8,value9
0,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 30 per hl,hectoliter,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 1.50 per 100 kg,kg,1.50,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,Iv f 30 per hl Acc f 1844 per hl,hectoliter,30,1844,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 10 per hl,hectoliter,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 169 per hl,hectoliter,169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,Iv f 80 per hl Acc f 353 per hl,hectoliter,80,353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 520 per hl,hectoliter,520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 400 per hl,hectoliter,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 100 per hl,hectoliter,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,Aruban Florin,NaN,f 150 per hl,hectoliter,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
spec.count()

advalorem_component1     40812
advalorem_component2      4225
advalorem_component3       119
advalorem_component4        29
advalorem_component5         5
currency_unit            74188
operator                 38636
text                    102573
unit                     59705
value1                   75126
value2                   13266
value3                    1106
value4                     318
value5                      37
value6                      10
value7                       2
value8                       2
value9                       1
dtype: int64

In [52]:
no_adv = spec[spec.advalorem_component1.isnull()]
misses = no_adv[no_adv.value1.isnull()].text.unique()

In [55]:
pattern = "\d+(?:.\d+)?( <=? )x( <=? )\d+(?:.\d+)?"
for miss in misses:
    m = re.search(pattern,miss)
    try:
        print(miss+": "+m.group(0))
    except AttributeError:
        pass

Examine AdValorem tariffs
-----

In [91]:
adv_dir = "D:/Users/cmarciniak/Documents/macmap/data/unctad/advalorem"
file = "008_000_2008.csv"
adv = pd.read_csv(os.path.join(adv_dir,file),dtype='object')

In [93]:
adv.PRODUCTCODE.str.len().value_counts()

6    5224
Name: PRODUCTCODE, dtype: int64

In [128]:
adv.head()

,Unnamed: 0,DATATYPE,FREQ,MAX_RATE,MIN_RATE,NBR_MFN_LINES,NBR_NA_LINES,NBR_PREF_LINES,NOMENCODE,OBS_VALUE,OBS_VALUE_MEASURE,PARTNER,PRODUCTCODE,REPORTER,SUM_OF_RATES,TARIFFTYPE,TIME_PERIOD,TOTALNOOFLINES
0,0,Reported,A,2,2,2,0,0,H2,2,SimpleAverage,000,010110,008,4,MFN,2008,2
1,1,Reported,A,10,10,4,0,0,H2,10,SimpleAverage,000,010190,008,40,MFN,2008,4
2,2,Reported,A,2,2,3,0,0,H2,2,SimpleAverage,000,010210,008,6,MFN,2008,3
3,3,Reported,A,10,2,12,0,0,H2,9.33333301544189,SimpleAverage,000,010290,008,112,MFN,2008,12
4,4,Reported,A,2,2,1,0,0,H2,2,SimpleAverage,000,010310,008,2,MFN,2008,1
